In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy as sp
from scipy import integrate
from scipy.interpolate import interp1d 
from classy import Class
import math as math
%matplotlib inline
import sys, platform, os
import re
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
import camb
from camb import model, initialpower

In [ ]:
#Fiducial cosmological parameters
c=3e5
hubble=0.678
omegab=0.022*pow(hubble,-2)
omegac=0.119*pow(hubble,-2)
om0=omegac+omegab
H00=100*hubble
Ass=2.14e-9
nss = 0.968
gamma=0.545

#table 1 from P&Y 1807.07076
euclid_data = np.loadtxt('tabula-1807.07076v2 porciani.txt')

#table 1 from P&Y has columns z, V, ng, b1, b2, bs, M
z_euclid = euclid_data[:,0]
v_euclid =  interp1d(z_euclid,10.0 * (1.0/(hubble**3)) * 1e9 *euclid_data[:,1])
ng_euclid = interp1d(z_euclid, 1e-3 * hubble**3 * euclid_data[:,2])
b1_euclid = interp1d(z_euclid, euclid_data[:,3])
b2_euclid = interp1d(z_euclid, euclid_data[:,4])
bs_euclid = interp1d(z_euclid, euclid_data[:,5])

#class, want to get the stuff all from camb though
cosmo = Class()
cosmo.compute()
background = cosmo.get_background()

#Set up the fiducial cosmology (CAMB)
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

#calculate results for these parameters
results = camb.get_results(pars)
background = camb.get_background(pars)

#Get matter power spectrum at z=zbin

zbin = 0.0
pars.set_matter_power(redshifts=[zbin], kmax=2.0)
#Use linear modelling
pars.NonLinear = model.NonLinear_none

#Non-Linear spectra (Halofit)
#pars.NonLinear = model.NonLinear_both

results = camb.get_results(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=2.0, npoints = 10000)
s8 = np.array(results.get_sigma8())

#Pm in Mpc^3 units, at z=0
Pmz0=interp1d(kh*hubble, (pk[0]/pow(hubble,3)))


#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))
#Define the comoving distance
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return sp.integrate.romberg(drdz,0,zc)

#Define the growth function in LCDM
def get_growth(zz):
    omz=om0*pow(1+zz,3)/(om0*pow(1+zz,3)+1-om0)
    return pow(omz,gamma)

#Get the growth factor 
def Dg_dz(zz):
    return get_growth(zz)/(1+zz)
def Dgz(zc):
    ans = sp.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(-ans)

#Power spectrum as fn of z 
def Pmz(kk,zc):    
    return pow(Dgz(zc),2)*Pmz0(kk)

In [ ]:
Z=1.0

Hu = cosmo.Hubble(Z)*(1/(1+Z))
H0 = cosmo.Hubble(0)
om_m0 = 0.308
om_m = om_m0 * (H0**2/Hu**2) *(1+Z)
f = cosmo.scale_independent_growth_factor_f(Z)
df = Hu* ((1/2)*(3*om_m -4)*f - f**2 + (3/2)*om_m)
dHu = H0**2 * (-(1/2)* (1+Z) * om_m0 + (1/(1+Z))**2 * (1-om_m0))
ddHu = H0**2 * ( (1/2)*Hu*(1+Z)*om_m0 + (1/(1+Z))**2 * 2 * Hu * (1-om_m0) )
chi = cosmo.angular_distance(Z)*(1+Z)
cap_L = 1
partdQ=0
B1 = 0.9 + 0.4 * Z
db1 = -0.4 * Hu * (1 + Z)
B2 = -0.704172 - 0.207993 * Z + 0.183023 * Z**2 - 0.00771288 * Z**3
b_e = -4
db_e=0
Q=(-1.0)*(-0.66 - 2.95 * Z + 1.59 * Z**2 - 0.40 * Z**3 + 0.04 * Z**4)
dQ= 0
gamma1 = Hu* (f * (b_e - 2*Q -(2*(1-Q)/(chi*Hu)) - (dHu/Hu**2)))
gamma2 = Hu**2 * (f*(3-b_e) + (3/2)*om_m*(2+b_e - f- 4*Q - (2*(1-Q)/(chi*Hu)) - (dHu/Hu**2) ))
partdb1 = 0 
fnl=0

#beta coeffs
beta1 = Hu**4 * ((9.0/4.0)*om_m**2 * (6- 2*f * (2*b_e - 4*Q - (4*(1-Q))/(chi*Hu) - (2*dHu)/(Hu**2)) - (2*df)/(Hu) + b_e**2 + 5*b_e - 8*b_e*Q + 4*Q + 16*Q**2 - 16*partdQ - 8*dQ/Hu + db_e/Hu + (2/(chi**2 * Hu**2))*(1-Q + 2*Q**2 - 2*partdQ)  - (2/(chi*Hu))*(3+2*b_e - 2*b_e*Q - 3*Q + 8*Q**2 - (3*dHu/Hu**2)*(1-Q) - 8*partdQ - 2*dQ/Hu) + (dHu/Hu**2)*(-7 -2*b_e + 8*Q + (3*dHu/Hu**2)) - (ddHu/Hu**3))  + ((3/2)*om_m * f)*(5 - 2*f*(4-b_e) + (2*df/Hu) + 2*b_e*(5 + ((2*(1-Q))/(chi*Hu))) - (2*db_e/Hu) - 2*b_e**2 + 8*b_e*Q - 28*Q - (14*(1-Q)/(chi*Hu)) - 3*dHu/Hu**2 + 4*(2-(1/(chi*Hu)))*(dQ/Hu) ) + ((3/2)*om_m*f**2)*(-2 + 2*f -b_e + 4*Q + (2*(1-Q)/(chi*Hu)) + (3*dHu/Hu**2) - (12/3)*fnl ) + f**2 * (12-7*b_e + b_e**2 + (db_e/Hu) + (b_e - 3)*(dHu/Hu**2)) - (3/2)*om_m*(df/Hu))
beta2 = Hu**4* ((9/2)*om_m**2 * (-1 + b_e - 2*Q - ((2*(1-Q))/(chi*Hu)) - (dHu/Hu**2) ) + 3*om_m*f*(-1 + 2*f -b_e + 4*Q + (2*(1-Q)/(chi*Hu)) + (3*dHu/Hu**2) )  + 3*om_m * f**2 * (-1+b_e-2*Q - ((2*(1-Q))/(chi*Hu)) - (dHu/Hu**2))  + 3*om_m*(df/Hu))
beta3 = Hu**3 * ((9/4)*om_m**2 *(f-2+2*Q) + (3/2)*om_m*f * (-2 -f*(-3 + f + 2*b_e - 3*Q - ((4*(1-Q))/(chi*Hu)) - (2*dHu/Hu**2)) - (df/Hu) + 3*b_e + b_e**2 - 6*b_e*Q + 4*Q + 8*Q**2 - 8*partdQ - 6*(dQ/Hu) + (db_e/Hu) + (2/(chi**2*Hu**2))*(1-Q +2*Q**2 - 2*partdQ) + (2/(chi*Hu))*(-1 -2*b_e + 2*b_e*Q + Q - 6*Q**2 + (3*dHu/Hu**2)*(1-Q) + 6*partdQ + 2*(dQ/Hu) ) - (dHu/Hu**2)*(3+2*b_e - 6*Q - (3*dHu/Hu**2)) - (ddHu/Hu**3))  + f**2 * (-3 + 2*b_e*(2+ ((1-Q)/(chi*Hu))) - b_e**2 + 2*b_e*Q - 6*Q - (db_e/Hu) - ((6*(1-Q))/(chi*Hu)) + 2*(1-(1/(chi*Hu)))*(dQ/Hu) ))
beta4 = (Hu**3 * ((9/2)*om_m*f * (-b_e + 2*Q +  (2*(1-Q)/(chi*Hu)) + (dHu/Hu**2))))
beta5 = (Hu**3 * ( 3* om_m * f * (b_e - 2*Q -(2*(1-Q)/(chi*Hu)) - (dHu/Hu**2))  ))
beta6 = Hu**2 * ((3/2) *om_m * (2-2*f + b_e - 4*Q - ((2*(1-Q))/(chi*Hu)) - (dHu/Hu**2)))
beta7 = Hu**2*(f*(3-b_e))
beta8= Hu**2*(3*om_m*f*(2-f-2*Q) + f**2 * (4 + b_e - b_e**2 + 4*b_e*Q - 6*Q - 4*Q**2 + 4*partdQ + 4*(dQ/Hu) -(db_e/Hu) - (2/(chi**2 * Hu**2))*(1-Q+2*Q**2 - 2*partdQ) - (2/(chi*Hu))*(3-2*b_e + 2*b_e*Q - Q - 4*Q**2 + ((3*dHu)/(Hu**2))*(1-Q) + 4*partdQ + 2*(dQ/Hu)) - (dHu/Hu**2)*(3-2*b_e+4*Q+((3*dHu)/(Hu**2))) + (ddHu/Hu**3)))
beta9 = (Hu**2 * ( -(9/2)*om_m*f ))
beta10 = (Hu**2 * (3 * om_m * f))
beta11 = Hu**2 * (3*om_m*(1+f) + 2*f - f**2*(-1+b_e-2*Q - ((2*(1+Q))/(chi*Hu)) - (dHu/Hu**2)))
beta12 = Hu**2*((3/2)*om_m * (-2 + B1 * (2+b_e-4*Q - ((2*(1-Q))/(chi*Hu)) - (dHu/Hu**2)) + (db1/Hu) + 2*(2-(1/(chi*Hu)))*partdb1) - f*(2+ B1*(f-3+b_e) + (db1/Hu)))
beta13 = ( (9.0/4.0)*om_m**2.0*Hu**2.0 )+ ( 1.5*om_m*f*Hu**2.0*(-(2.0*f)+(2.0*b_e)-(6.0*Q) -((4.0*(1.0-Q))/(chi*Hu)) -((3.0*dHu)/Hu**2.0) ) )+ ( f**2.0*Hu**2.0*(5.0-b_e) )
beta14 = Hu* ( -(3/2)*om_m*B1 )
beta15 = Hu*2*f**2 
beta16 = Hu*(f*(B1 * (f+b_e-2*Q - ((2*(1-Q))/(chi*Hu)) - (dHu/Hu**2)) + (db1/Hu) + 2*(1- (1/(chi*Hu)))*partdb1 ))
beta17 = Hu*(-(3/2)*om_m*f)
beta18 = Hu* ( (3/2)*om_m*f - f**2 * (3 - 2*b_e + 4*Q + ((4*(1-Q))/(chi*Hu)) + (3*dHu/Hu**2)) )
beta19 = Hu * (f* (b_e - 2*Q - ((2*(1-Q))/(chi*Hu)) - (dHu/Hu**2)))

def Pm(i):
    kk = k[i]
    return Pmz(kk,Z)

In [ ]:
#k = {1:k1,2:k2,3:k3,"theta":theta}
#mu = {1:mu1,2:mu2,3:mu3}

# def get_mus(self,MU_1,PHI,k):
#     mu = {1:MU_1}
#     mu[2]=mu[1]*np.cos(k["theta"]) + np.sqrt(1.0-mu[1]**2) * np.sin(k["theta"])*np.cos(PHI)
#     mu[3] = - (k[1] / k[3]) * mu[1] - (k[2] / k[3]) * mu[2]
#     return mu

def get_theta(k1,k2,k3):
    return np.arccos(0.5 * (k3**2 - (k1**2 + k2**2))/(k1 * k2))

def get_mus(MU_1,PHI,k):
    mu = {1:MU_1}
    mu[2]=mu[1]*np.cos(k["theta"]) + np.sqrt(1.0-mu[1]**2) * np.sin(k["theta"])*np.cos(PHI)
    mu[3] = - (k[1] / k[3]) * mu[1] - (k[2] / k[3]) * mu[2]
    return mu


def get_cosinus(i,j,k):
    assert i!=j
    assert j!=l
    assert i in [1,2,3]
    assert j in [1,2,3]
    assert l in [1,2,3]
    
    return 0.5 * (k[l]**2 - k[i]**2 - k[j]**2) / (k[i] + k[j])
    

def E(i,j,l,k):
    consinus = get_cosinus(i,j,k)
    return ((k[i]**2 * k[j]**2) / (k[l]) ) * (3 + 2 * cosinus * (k[i]/k[j] + k[j]/k[i]) + cosinus**2  )

def F(i,j,l,k):
    consinus = get_cosinus(i,j,k)
    return 10/7 + cosinus * (k[i]/k[j] + k[j]/k[i]) + (1 - 3/7)*cosinus**2
    
def G(i,j,l,k):
    consinus = get_cosinus(i,j,k)
    return 6/7 + cosinus * (k[i]/k[j] + k[j]/k[i]) + (2 - 6/7) * cosinus**2 

def KN1(i,mu,B1):
    return B1 + f * mu[i]**2

def KGR1(i,mu,gamma1,gamma2):
    return (gamma2/k[i])**2 + (gamma1/k[i])*1j*mu[i]


#this is where Eline's gonna fuck up 
def KN2(i,j,l,k,mu,B1,B2):
    cosinus = get_cosinus(i,j,k)
    F = F(i,j,l,k)
    G = G(i,j,l,k)
    return B1 * F + B2 + f * G + mu[l]**2 - 4/7 * (B1 - 1) * (cosinus**2 - 1/3) +\
    f**2 * (mu[i] * mu[j])/(k[i] * k[j]) * (mu[i] * k[i] + mu[j] * k[j])**2 + B1 * f/(k[i] * k[j]) *\
    ((mu[i]**2 + mu[j]**2) * k[i] * k[j] + mu[i] * mu[j] * (k[i]**2 + k[j]**2))


def KGR2(i,j,l,k,mu,beta1,beta2,beta3,beta4,beta5,beta6,beta7,beta8,beta9,beta10,beta11,beta12,beta13,beta14,beta15,beta16,beta17,beta18,beta19):
    cosinus = get_cosinus(i,j,k)
    E = E(i,j,l,k)
    F = F(i,j,l,k)
    G = G(i,j,l,k)
    return 1/(k[i]**2 * k[j]**2) * (beta1 + E * beta2 + 1j * (mu[i] * k[i] + mu[j] * k[j]) * beta3 + mu[l] * k[l] * (beta4 + E * beta5) + (k[i]**2 + k[j]**2)/(k[l]**2) * (F * beta6 + G * beta7) +\
    (mu[i] * k[i] * mu[j] * k[j]) * beta8 * mu[l]**2 * k[l]**2 * (beta9 + E * beta10) + k[i] * k[j] * cosinus * beta11 + (k[i]**2 + k[j]**2) * beta12 + (mu[i]**2 * k[i]**2 + mu[j]**2 + k[j]**2) * beta13 + 1j *\
    ((mu[i] * k[i]**3 + mu[j] * k[j]**3) * beta14 + (mu[i] * k[i] + mu[j] * k[j]) * k[i] * k[j] * cosinus * beta15 + k[i] * k[j] * (mu[i] * k[j] + mu[j] * k[i]) * beta16 + (mu[i]**3 * k[i]**3 + mu[j]**3 * k[j]**3) * beta17 +\
    mu[i] * mu[j] * k[i] * k[j] * (mu[i] * k[i] + mu[j] * k[j]) * beta18 + mu[l] * (k[i]**2 * k[j]**2)/k[l] * G * beta19 ) )




def B_full(k,mu,B1,B2,gamma1,gamma2,beta1,beta2,beta3,beta4,beta5,beta6,beta7,beta8,beta9,beta10,beta11,beta12,beta13,beta14,beta15,beta16,beta17,beta18,beta19):
    perm1 = (KN1(1,mu,B1)+KGR1(1,mu,gamma1,gamma2)) * (KN1(2,mu,B1)+KGR1(2,mu,gamma1,gamma2)) * (KN2(1,2,3,k,mu,B1,B2) + KGR2(1,2,3,k,mu,beta1,beta2,beta3,beta4,beta5,beta6,beta7,beta8,beta9,\
            beta10,beta11,beta12,beta13,beta14,beta15,beta16,beta17,beta18,beta19)) * Pm(1) * Pm(2) 
    perm2 = (KN1(1,mu,B1)+KGR1(1,mu,gamma1,gamma2)) * (KN1(3,mu,B1)+KGR1(3,mu,gamma1,gamma2)) * (KN2(1,3,2,k,mu,B1,B2) + KGR2(1,3,2,k,mu,beta1,beta2,beta3,beta4,beta5,beta6,beta7,beta8,beta9,\
            beta10,beta11,beta12,beta13,beta14,beta15,beta16,beta17,beta18,beta19)) * Pm(1) * Pm(3) 
    perm3 = (KN1(2,mu,B1)+KGR1(2,mu,gamma1,gamma2)) * (KN1(3,mu,B1)+KGR1(3,mu,gamma1,gamma2)) * (KN2(2,3,1,k,mu,B1,B2) + KGR2(2,3,1,k,mu,beta1,beta2,beta3,beta4,beta5,beta6,beta7,beta8,beta9,\
            beta10,beta11,beta12,beta13,beta14,beta15,beta16,beta17,beta18,beta19)) * Pm(2) * Pm(3) 
    return perm1 + perm2 + perm3

In [ ]:

k1_bins = np.arange(0.1,1,1/20)
k2_bins = np.arange(0.1,1,1/20)
k3_bins = np.arange(0.1,1,1/20)


for k1 in k1_bins:
    for k2 in k2_bins[k2_bins<=k1]:
        for k3 in k3_bins[k3_bins<=k2]:
            k1=0.1
            k2=0.1
            k3=0.5001
            k = {1:k1,2:k2,3:k3,"theta":get_theta(k1,k2,k3)}
            

In [ ]:
kmin = 0.01
kmax = 0.1
deltak = (kmax-kmin)/20

for k1 in np.arange(kmin, kmax, deltak):
    for k2 in np.arange(k1,kmax,deltak):
        for k3 in np.arange(max(kmin,abs(k1-k2)),kmax,deltak):
            if (k1+k2>k3 and k1+k3>k2 and k2+k3>k1)==True:
                k = {1:k1,2:k2,3:k3,"theta":get_theta(k1,k2,k3)}
                mu = get_mus(-1,0.9,k)
                bisp = B_full(k,mu,B1,B2,gamma1,gamma2,beta1,beta2,beta3,beta4,beta5,beta6,beta7,beta8,beta9,\
                beta10,beta11,beta12,beta13,beta14,beta15,beta16,beta17,beta18,beta19)
                